In [ ]:
from dotenv import load_dotenv

load_dotenv()


True

In [42]:
import os
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")
TAVILY_KEY = os.getenv("TAVILY_API_KEY")


In [30]:
model = ChatOpenAI(model="gpt-4o-mini", api_key=OPEN_AI_API_KEY)

prompt = "What is the weather now in Hungary, Debrecen?"

response = model.invoke([HumanMessage(content=prompt)])
print(f"Usage metadata: {response.usage_metadata}")
print(f"Response: {response.content}")

print(f"\nDEBUG: {response}")


Usage metadata: {'input_tokens': 19, 'output_tokens': 40, 'total_tokens': 59}
Response: I'm unable to provide real-time weather updates. To find the current weather in Debrecen, Hungary, I recommend checking a reliable weather website or app for the most accurate and up-to-date information.

DEBUG: content="I'm unable to provide real-time weather updates. To find the current weather in Debrecen, Hungary, I recommend checking a reliable weather website or app for the most accurate and up-to-date information." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 19, 'total_tokens': 59, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None} id='run-c33abcba-80a4

# Using the Tavily tool standalone


In [43]:
tavily = TavilySearchResults(max_results=2, verbose=True, include_answer=True)
search_results = tavily.invoke("What is the weather now in Hungary, Debrecen?")
search_results


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Debrecen', 'region': 'Debrecen', 'country': 'Hungary', 'lat': 47.5333, 'lon': 21.6333, 'tz_id': 'Europe/Budapest', 'localtime_epoch': 1733144903, 'localtime': '2024-12-02 14:08'}, 'current': {'last_updated_epoch': 1733144400, 'last_updated': '2024-12-02 14:00', 'temp_c': 3.3, 'temp_f': 37.9, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 3.6, 'wind_kph': 5.8, 'wind_degree': 199, 'wind_dir': 'SSW', 'pressure_mb': 1025.0, 'pressure_in': 30.27, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 100, 'cloud': 75, 'feelslike_c': 1.9, 'feelslike_f': 35.3, 'windchill_c': 5.8, 'windchill_f': 42.5, 'heatindex_c': 6.8, 'heatindex_f': 44.2, 'dewpoint_c': 2.2, 'dewpoint_f': 35.9, 'vis_km': 6.0, 'vis_miles': 3.0, 'uv': 0.4, 'gust_mph': 4.9, 'gust_kph': 7.9}}"}, {'url': 'https://hungaryweather.org/debrecen', 'content': "6°C Temperature: 5.5

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Debrecen', 'region': 'Debrecen', 'country': 'Hungary', 'lat': 47.5333, 'lon': 21.6333, 'tz_id': 'Europe/Budapest', 'localtime_epoch': 1733144903, 'localtime': '2024-12-02 14:08'}, 'current': {'last_updated_epoch': 1733144400, 'last_updated': '2024-12-02 14:00', 'temp_c': 3.3, 'temp_f': 37.9, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 3.6, 'wind_kph': 5.8, 'wind_degree': 199, 'wind_dir': 'SSW', 'pressure_mb': 1025.0, 'pressure_in': 30.27, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 100, 'cloud': 75, 'feelslike_c': 1.9, 'feelslike_f': 35.3, 'windchill_c': 5.8, 'windchill_f': 42.5, 'heatindex_c': 6.8, 'heatindex_f': 44.2, 'dewpoint_c': 2.2, 'dewpoint_f': 35.9, 'vis_km': 6.0, 'vis_miles': 3.0, 'uv': 0.4, 'gust_mph': 4.9, 'gust_kph': 7.9}}"},
 {'url': 'https://hungaryweather.org/debrecen',
  'content': "6°C Temperature

# Using tools with a react agents


In [7]:
agent = create_react_agent(model, [tavily])

# prompt = "What is OOP?"
prompt = "What is the weather now in Hungary, Debrecen?"

response = agent.invoke({"messages": [HumanMessage(content=prompt)]})

response["messages"]


content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Debrecen\', \'region\': \'Debrecen\', \'country\': \'Hungary\', \'lat\': 47.5333, \'lon\': 21.6333, \'tz_id\': \'Europe/Budapest\', \'localtime_epoch\': 1733127193, \'localtime\': \'2024-12-02 09:13\'}, \'current\': {\'last_updated_epoch\': 1733126400, \'last_updated\': \'2024-12-02 09:00\', \'temp_c\': 0.3, \'temp_f\': 32.5, \'is_day\': 1, \'condition\': {\'text\': \'Freezing fog\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/260.png\', \'code\': 1147}, \'wind_mph\': 2.5, \'wind_kph\': 4.0, \'wind_degree\': 186, \'wind_dir\': \'S\', \'pressure_mb\': 1027.0, \'pressure_in\': 30.33, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 100, \'cloud\': 100, \'feelslike_c\': -0.7, \'feelslike_f\': 30.7, \'windchill_c\': 3.0, \'windchill_f\': 37.3, \'heatindex_c\': 3.6, \'heatindex_f\': 38.4, \'dewpoint_c\': 1.8, \'dewpoint_f\': 35.2, \'vis_km\': 0.2, \'vis_miles\': 0.0, \'uv\': 0.4, \'gust_mph\

[HumanMessage(content='What is the weather now in Hungary, Debrecen?', additional_kwargs={}, response_metadata={}, id='6083f7b8-86fb-4646-828f-64ba3711eecd'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XegzIfPvNwSjJuLea6GJwDjr', 'function': {'arguments': '{"query":"current weather in Debrecen, Hungary"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 91, 'total_tokens': 116, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_3de1288069', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-3d07a0d6-7ba9-4ac6-b194-55ab3e67e411-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Debrecen, Hu

In [8]:
print(response["messages"][-1].content)


The current weather in Debrecen, Hungary is as follows:

- **Temperature:** 0.3°C (feels like -0.7°C)
- **Condition:** Freezing fog
- **Humidity:** 100%
- **Wind:** 2.5 mph (4.0 kph) from the South
- **Visibility:** 0.2 km

For more details, you can check the weather [here](https://www.weatherapi.com/).


# Trying to do math without a calculator and only with an LLM


In [37]:
agent = create_react_agent(model, [tavily])

A = 12311231
B = 15411414223

prompt = f"Multiply {A} with {B}. Respond only with the number, without any extra content or punctuation"

response = agent.invoke({"messages": [HumanMessage(content=prompt)]})

response["messages"][-1].content


content='[{"url": "https://aerobasegroup.com/part-number/12311231_01-128-5509", "content": "12311231 Automotive Eng Maintenance Stand | AeroBase Group, Inc. Parts Lookup \xa0 Parts Search Parts by End Item Commercial Aircraft Parts Parts by Category Parts \xa0 Parts Lookup \xa0 Parts Search Parts by End Item Commercial Aircraft Parts Parts by Category The name and number of those parts furnished with the item of supply that have not been specified elsewhere.AKYN Cross Reference Parts Product Condition Codes | FN | Factory New. Parts manufactured by then OEM and includes manufacturer certification. | NE | New. Parts manufactured by the OEM. Parts Parts by End Item Parts by Series Commercial Aircraft Parts Two Russian fighters intercepted a pair of B-52H Stratofortresses near the Baltic Sea on Monday, a U.S. defense official confirmed. Recent parts by keyword  "}, {"url": "https://www.chegg.com/homework-help/questions-and-answers/find-2-x-2-stochastic-matrix-identity-19311931-12311231-st

'18915541124755385353'

In [38]:
result = int(response["messages"][-1].content)
correct_response = A * B
print(correct_response == result)
print(f"correct result: {correct_response}, returned result: {result}")


False
correct result: 189733480536038513, returned result: 18915541124755385353


# Introducing our own tool to multiply numbers


In [18]:
from langchain_core.tools import tool


@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b


In [39]:
agent = create_react_agent(model, [tavily, multiply])
response = agent.invoke({"messages": [HumanMessage(content=prompt)]})

response["messages"]


[HumanMessage(content='Multiply 12311231 with 15411414223. Respond only with the number, without any extra content or punctuation', additional_kwargs={}, response_metadata={}, id='29a5557a-037d-4486-90bd-80ae6af49cd2'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yUtmBiNOuYgi15AsvU4dL1Ze', 'function': {'arguments': '{"a":12311231,"b":15411414223}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 124, 'total_tokens': 146, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-95ebf06c-72dd-48e3-84e1-b79eaf4204f1-0', tool_calls=[{'name': 'multiply', 'args': {'a': 12311231, 'b': 15411414223},

In [41]:
result = int(response["messages"][-1].content)
print(correct_response == result)
print(f"correct result: {correct_response}, returned result: {result}")


True
correct result: 189733480536038513, returned result: 189733480536038513
